In [2]:
import numpy as np
import ctypes as ct
import glob, os

import os.path
from os import path

import tkinter
from tkinter import messagebox

import run_routing

import CreationParam
import ExecProg
import GraphRoutes
import GraphPoints

# Bokeh libraries
from bokeh.io import output_file
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.models.widgets import Panel, Tabs, Toggle, TextInput
from bokeh.layouts import column, row  , layout
from bokeh.models import CustomJS, Div, Button
from bokeh.events import ButtonClick
from bokeh.io import curdoc
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Select
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
from bokeh.models.widgets import Paragraph

chemin = "/home/nathan/TAL/Routage/Sources/NOISY_PAES/"
cheminMeteo = "/home/nathan/TAL/grib/"
cheminPolar = "/home/nathan/TAL/polar/"

nbExec = 0


#******************************************************************************************************
#Création graph 

# On choisis les outils de la barre d'outils
select_tools = ['box_zoom']


# On créer la figure
fig = figure(title='Graph meilleurs chemins',
             plot_height=750, plot_width=750,
             x_axis_label='Longitude', y_axis_label='Latitude', 
             toolbar_location="right",tools=select_tools, 
             match_aspect=True)

'''    
# emplacement légende
fig.legend.location = 'top_left'
'''

#********************************************************************************************************
#Création du 2nd graph

# On créer la figure
fig1 = figure(title='Graph isofront',
             plot_height=400, plot_width=400,
             x_axis_label='Time', y_axis_label='Stress', 
             toolbar_location="right",tools=select_tools)


'''
# emplacement légende
fig1.legend.location = 'top_center'
'''


#*******************************************************************************************************
#Menu

#differente technique pour récuperer le nom des fichiers sans avoir besoin de passer par des fichier .txt
#ne fonctionne pas


#on créer les widgets

'''
laD = TextInput(value="44.31725", title="Lat depart :")
loD= TextInput(value="-1.33679", title="Long depart :")
laA = TextInput(value="47.54304", title="Lat arrivee :")
loA = TextInput(value="-6.40641", title="Long arrivee :")
date = TextInput(value="20151005", title="Date :")
stime = TextInput(value="185544", title="Stime :")
'''

laD = TextInput(value="47.29891", title="Lat depart :")
loD= TextInput(value="-3.57457", title="Long depart :")
laA = TextInput(value="44.28568", title="Lat arrivee :")
loA = TextInput(value="-5.32721", title="Long arrivee :")
date = TextInput(value="20151005", title="Date :")
stime = TextInput(value="185544", title="Stime :")

#on recupere les noms des fichiers polar disponible : 
tabPolar = []
for file in glob.glob(cheminPolar+"*.xml"):  
    file = file.replace(cheminPolar+'','')
    file = file.replace('.xml','')
    
    tabPolar.append(file)
    
polar = Select(title="Polar:", value=tabPolar[0], options=tabPolar)   


#on récupere les noms des fichiers meteo disponible : 
tabWeather = []
for file in glob.glob(cheminMeteo+"gribfile/*.grb"): 
    file = file.replace(cheminMeteo+'gribfile/','')
    file = file.replace('.grb','')
    
    tabWeather.append(file)
    
weather = Select(title="Weather:", value=tabWeather[0], options=tabWeather)   



button = Button(label="Lancer programme", button_type="success")

button1 = Button(label="Modifier params secondaires")

par = Paragraph(text="""""",style={'color': 'red'})


#les widgets "secondaires" sont désactivé au lancement de l'application
seed = TextInput(value="5", title="Seed :",disabled=True)
noise = TextInput(value="0.0", title="Noise :",disabled=True)
gens = TextInput(value="4000", title="Gens :",disabled=True)
alpha = TextInput(value="100", title="Alpha :",disabled=True)
depth = TextInput(value="4", title="depth :",disabled=True)


#*******************************************************************************************************
#CALLBACKS

def creationParam():
    
    #on récupere les differentes valeurs 
    Seed = "seed 5"
    start = "start "+laD.value+" "+loD.value
    finish = "finish "+laA.value+" "+loA.value
    Date = "date "+date.value
    Stime = "stime "+stime.value
    
    Polar = "polar ../"+polar.value+".dat"
    Weather = "weather ../"+weather.value+".isoc" 
    
    Noise = "noise 0.0"
    Gens = "gens 4000"
    Alpha = "alpha 100"
    Depth = "depth 4"
    outputfile = "outputfile output.out"
    
    CreationParam.creationParam(Seed,start,finish,Date,Stime,Polar,Weather,Noise,Gens,Alpha,Depth,outputfile)
    
    
def lancerProg():
    
    #dans un premier temps on vérifie l'existence des fichier .dat et .isco
    #si les fichiers n'existent pas, on utilise l'executable  sur les fichier .xml et .grib
    #on lance dans un premier temps les 2 exec permettant de creer les ficher .dat et .isoc
   
    ExecProg.polar(polar.value)
    ExecProg.grib(weather.value)

    '''
    #CARTO 
    #dans un premier temps , on lance le programme run-routing.py pour generer les fichier nécéssaires
    
    longmax = max(float(laD.value),float(laA.value)) + 10
    longmin = min(float(laD.value),float(laA.value)) - 10
    latmax = max(float(loD.value),float(loA.value)) + 10
    latmin = min(float(loD.value),float(loA.value)) - 10 
      
    #Version 1
    #plus besoin d'appeller cette fonction : trop de ralentisement du programme
    #run_routing.generate_map(10,'Carto/SRC/lines_data.json', 'world_map.dat')
    
    #fonction pour generer le fichier ""
    #run_routing.generate_borned_map(longmax, longmin, latmax, latmin) 
    '''
    
    ExecProg.calculRoutes()

    
def MiseAJourGraph(val):   
    
    #*******************************************************************************************
    #Fond de carte :    
    
    #*********************************************************************************************
    #PLOT 1
       
    #suite a l'exec du programme c, on va chercher les nouvelles valeurs dans dans le fichier data
    #on met a jour la variable "source" pour mettre a jour le graph
    os.chdir(chemin)
             
    ''' 
    #version avec module : 
    
    GraphRoutes.routePrincipale()
    GraphRoutes.autreRoutes()
 

    GraphPoints.routePrincipale()
    GraphPoints.autreRoutes()
    '''
    
    #*********************************************************************************************
    #PLOT 1
    


    #suite a l'exec du programme c, on va chercher les nouvelles valeurs dans dans le fichier data
    #on met a jour la variable "source" pour mettre a jour le graph
    os.chdir(chemin)
             
    #le meilleur chemin :         
    tailleList = 0         
    with open("noisy_iso.dat", "r") as ins:
        arrayLat = []
        arrayLong = []
        arrayDist = []
        arrayTime = []
        for line in ins:
            cdc = line
            maListe = cdc.split(" ")
            tailleList = tailleList+1
            #on lit seulement les lignes qui contiennent des données
            if maListe[0] == '\n':
                break
            elif maListe[0] != '##':
                for j in range(len(maListe)):
                    #print("value : " + maListe[j])
                    maListe[j] = float(maListe[j]) 
                arrayLat.append(maListe[8])
                arrayLong.append(maListe[9])
                arrayDist.append(maListe[5])
                arrayTime.append(maListe[7])         
      
    with open("noisy_isofront.dat", "r") as ins:
        arrayStress = []
        for line in ins:
            cdc = line
            maListe = cdc.split(" ")       
            for i in range(len(maListe)):
                try:
                    maListe[i] = float(maListe[i])
                except: 
                    pass
            for i in range (1,tailleList-2):
                arrayStress.append(maListe[1])   

                
    
    #if premiere exec : création source, puis création de la ligne 
    if val < 1 :
        sourceMeilleurCheminLigne = ColumnDataSource(data=dict(x=arrayLong, y=arrayLat, dist=arrayDist, 
                                         time=arrayTime, stress=arrayStress)) 

        #on creer la ligne du meilleur chemin
        fig.line('x','y',source=sourceMeilleurCheminLigne, 
                 color='blue', line_width=3,
                 legend='Meilleur chemin')
    
    #sinon : on met juste a jour la source
    else:
        sourceMeilleurCheminLigne.data = dict(x=arrayLong, y=arrayLat, 
                                              dist=arrayDist, time=arrayTime, stress=arrayStress) 



    
    #autres chemins 
    tabStress = []
    with open("noisy_front.dat", "r") as ins:
        num = 1
        for line in ins:
            cdc = line
            maListe = cdc.split(" ")
            for j in range(len(maListe)):
                try:
                    maListe[j] = float(maListe[j])
                except:
                    pass
                tabStress.append(maListe[1])  
    
    
    tabChemin = []
    val = 0
    with open("noisy_ps.dat", "r") as ins:
        arrayDist = []
        arrayTime = []
        arrayLat = []
        arrayLong = []
        arrayStress = []
        
        #Je reset toutes les sources pour effacer les anciens chemins 
        for i in range(len(tabChemin)):           
            tabChemin[i].data = dict(x=arrayLong, y=arrayLat, dist=arrayDist,time=arrayTime) 
        
        
        for line in ins:
            cdc = line
            maListe = cdc.split(" ")
            #if maListe est vide : ligne blanche
            if maListe[0] == '\n':
                #si array deja vide on fait rien
                #sinon on affiche la ligne et on reset les 2 array
                if len(arrayLat)!=0 and len(arrayLong)!=0 :                        
                    
                    for i in range(len(arrayLat)):                    
                        arrayStress.append(tabStress[val])
                        
                    val = val+1
                    
                           
                    s = ColumnDataSource(data=dict(x=arrayLong, y=arrayLat, dist=arrayDist,
                                               time=arrayTime, stress=arrayStress))  
                    tabChemin.append(s)
                    
                         
                    arrayDist = []
                    arrayTime = []
                    arrayLat = []
                    arrayLong = []
                    arrayStress = []  
                    
            #on lit seulement les lignes qui contiennent des données
            if maListe[0] != '##' and maListe[0] != '\n':          
                for j in range(len(maListe)):
                    maListe[j] = float(maListe[j]) 
                #on extrait les x et y 
                arrayLat.append(maListe[8])
                arrayLong.append(maListe[9])
                arrayDist.append(maListe[5])
                arrayTime.append(maListe[7])
                
    
    #on trace les chemins en parcourant le tableau             
    for i in range(len(tabChemin)):
        nom = 'line'+str(i)
        fig.line('x', 'y', source=tabChemin[i], color='black', line_width=0.5, name=nom)            
                

    
    '''
    
    
    #***************************************************************************************************
    #PLOT 2
      
    #le meilleur chemin :
    with open("noisy_isofront.dat", "r") as ins:
        arrayTime = []
        arrayStress = []
        for line in ins:
            cdc = line
            maListe = cdc.split(" ")       
            for i in range(len(maListe)):
                try:
                    maListe[i] = float(maListe[i]) 
                except : 
                    pass
                arrayTime.append(maListe[0])
                arrayStress.append(maListe[1])
     
    
    if val : 
        sourceMeilleurCheminPoint = ColumnDataSource(data=dict(x=arrayTime, y=arrayStress)) 

        # Draw the coordinates as circles
        fig1.circle('x', 'y',source=sourceMeilleurCheminPoint,
                   color='blue', size=10, alpha=0.5,
                   legend='Meilleur chemin')
    else:
        sourceMeilleurCheminPoint.data = dict(x=arrayTime, y=arrayStress)


    
    tabPoint = []
    #autres chemins
    num=0
    with open("noisy_front.dat", "r") as ins:
        arrayTime = []
        arrayStress = []
    
        
        #Je reset toutes les sources pour effacer les anciens chemins 
        for i in range(len(tabPoint)):           
            tabPoint[i].data = dict(x=arrayTime, y=arrayStress) 
        
        for line in ins:
            cdc = line
            maListe = cdc.split(" ")
            for j in range(len(maListe)-1):
                    try :
                        maListe[j] = float(maListe[j]) 
                    except:
                        pass
                    arrayTime.append(maListe[0])
                    arrayStress.append(maListe[1])                    
                    
                    s = ColumnDataSource(data=dict(x=arrayTime, y=arrayStress))  
                    tabPoint.append(s)
                    
                    arrayTime = []
                    arrayStress = []
                    
                    
    #on place les points en parcourant le tableau             
    for i in range(len(tabPoint)):
        nom = 'point'+str(i)
        fig1.circle('x', 'y',source=tabPoint[i],
                    color='black', size=5, alpha=0.5, name=nom)              
                    

    '''
    
    
    
#****************************************************************************************************
    
def program():
    
    global nbExec
    
    nbExecLocal = nbExec 
    nbExec = nbExec + 1
    
    creationParam()
    lancerProg()
    MiseAJourGraph(nbExecLocal)
    
    
def hide():
    if seed.disabled==True :
        seed.disabled=False
        noise.disabled=False
        gens.disabled=False
        alpha.disabled=False
        depth.disabled=False
        button1.label="Bloquer params secondaires"
    else :
        seed.disabled=True
        noise.disabled=True
        gens.disabled=True
        alpha.disabled=True
        depth.disabled=True
        button1.label="Modifier params secondaires"

#******************************************************************************************************    

button.on_click(program) 
button1.on_click(hide)

#*********************************************************************************************************
#Organisation de la page


# Format the tooltip
tooltips = [
            ('dist to dest', '@dist'),
            ('time to dest', '@time'),
            ('stress global', '@stress'),
           ]

tooltips2 = [
            ('stress', '@y'),
            ('time', '@x'),
           ]

# Add the HoverTool to the figure
fig.add_tools(HoverTool(tooltips=tooltips))
fig1.add_tools(HoverTool(tooltips=tooltips2))


controls1 = [laD,laA,date,polar,button,par,button1,seed,noise,gens,alpha,depth]
controls2 = [loD,loA,stime, weather,fig1]

inputs1 = column(*controls1, width=400, height=500)
inputs2 = column(*controls2, width=400, height=500)

controls3 = [inputs1,inputs2, fig]

inputs3 = row(*controls3, width=100, height=100)

l = layout([
    [inputs3],
], sizing_mode="scale_both")


curdoc().add_root(l)
curdoc().title = "Application routage voilier"